In [1]:
# backend.py
# --- Step 1: Install necessary libraries ---
!pip install fastapi uvicorn pyngrok diffusers transformers accelerate torch Pillow

import torch
from fastapi import FastAPI
from pydantic import BaseModel
from diffusers import StableDiffusionPipeline
import base64
from io import BytesIO
from PIL import Image
import uvicorn
from pyngrok import ngrok, conf
import threading
import os

# --- Optional: Add your ngrok authtoken if you have one ---
# Replace "YOUR_NGROK_AUTHTOKEN" with your actual token from https://dashboard.ngrok.com/get-started/your-authtoken
conf.get_default().auth_token = "31QAivzbppt5jb8ozM3TkemWwlN_6BKgSbrv5qYALrgK7pZ3Q"

print("Starting backend server...")

# ---- Step 2: Load the Stable Diffusion model AND your LoRA ----
print("Loading Stable Diffusion model...")
# Define the base model and your LoRA model from the Hugging Face Hub
base_model_id = "runwayml/stable-diffusion-v1-5"
lora_model_id = "nikhiljose7/lora-sdv1-5-minnal-murali"

# Use float16 for faster inference and less memory usage.
pipe = StableDiffusionPipeline.from_pretrained(base_model_id, torch_dtype=torch.float16)

print(f"Loading LoRA weights from: {lora_model_id}")
# Load the LoRA weights into the base model
pipe.load_lora_weights(lora_model_id)

# Move the model to the GPU if available.
if torch.cuda.is_available():
    print("Moving model to GPU...")
    pipe = pipe.to("cuda")
else:
    print("CUDA not available. Using CPU (this will be very slow).")
    pipe = pipe.to("cpu")

# ---- Step 3: Set up the FastAPI app ----
app = FastAPI()

# Define the request body structure.
# The key must be "caption".
class Prompt(BaseModel):
    caption: str

@app.get("/")
def read_root():
    return {"status": "ok"}

# Define the image generation endpoint.
@app.post("/generate")
def generate_image(data: Prompt):
    """
    Generates an image from a text caption.
    """
    # --- MODIFIED: Add the trigger word to the prompt ---
    # This is crucial to activate your LoRA model's concept.
    trigger_word = "Minnal Murali"
    prompt_with_trigger = f"{trigger_word}, {data.caption}"

    print(f"Received caption: {data.caption}")
    print(f"Using full prompt: {prompt_with_trigger}")

    try:
        # Generate the image using the modified prompt.
        image = pipe(prompt_with_trigger).images[0]

        # Convert the PIL image to a base64 string to send via JSON.
        buffered = BytesIO()
        image.save(buffered, format="PNG")
        img_str = base64.b64encode(buffered.getvalue()).decode("utf-8")

        print("Image generated successfully.")
        return {"image_base64": img_str}
    except Exception as e:
        print(f"An error occurred: {e}")
        return {"error": str(e)}, 500


# ---- Step 4: Run the server using Uvicorn and Ngrok ----
# We run Uvicorn in a separate thread so that we can start ngrok in the main thread.
def run_app():
    uvicorn.run(app, host="0.0.0.0", port=8000)

# Open a public tunnel to the local server
public_url = ngrok.connect(8000)
print("------------------------------------------------")
print(f"✅ Public API URL: {public_url}")
print("------------------------------------------------")
print("Copy this URL and paste it into the API_URL variable in your frontend.py script.")


# Start the FastAPI server in the background.
thread = threading.Thread(target=run_app, daemon=True)
thread.start()

# Keep the main thread alive to keep ngrok running.
try:
    while True:
        pass
except KeyboardInterrupt:
    print("Shutting down server and ngrok tunnel.")
    ngrok.disconnect(public_url)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 53.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 55.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 54.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 54.5 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalli

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


model_index.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

Fetching 15 files:   0%|          | 0/15 [00:00<?, ?it/s]

preprocessor_config.json:   0%|          | 0.00/342 [00:00<?, ?B/s]

...

  0%|          | 0/50 [00:00<?, ?it/s]

Image generated successfully.
INFO:     34.127.33.101:0 - "POST /generate HTTP/1.1" 200 OK
Received caption: Minnal Murali super hero red costume
Using full prompt: Minnal Murali, Minnal Murali super hero red costume


  0%|          | 0/50 [00:00<?, ?it/s]

Image generated successfully.
INFO:     34.127.33.101:0 - "POST /generate HTTP/1.1" 200 OK
Shutting down server and ngrok tunnel.


PyngrokNgrokURLError: ngrok client exception, URLError: [Errno 111] Connection refused